# Intermediate Lesson on Geospatial Data 

## Structured Query Language (SQL)

<strong>Lesson Developers:</strong> Jayakrishnan Ajayakumar, Shana Crosson, Mohsen Ahmadkhani

#### Part 3 of 5

In [86]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display, clear_output
from ipywidgets import interactive, Textarea, HBox, Button, Layout
import ipywidgets as widgets
import sqlite3
import pandas as pd


import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
# sys.path.append('supplementary')
import hourofci
try:
    import os
    os.chdir('supplementary')
except:
    pass

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

## But how do we talk to the DBMS?
We now know that DBMS acts as an interface between us and the database and helps to convert our request so that the database can understand it. We use a special language called **SQL** to fetch data from the database. 

>**Structured Query Language (SQL)** is a programming language designed to get information out of and put it into a relational database.

<img src = "supplementary/images/sql_funny.png" width = "600px">


Below you see two databases and corresponding tables. Use the dropdowns to select the database and tables. What do you notice about the data types and data content in each table?

In [87]:
import displaydatabases
from questiondisplay import QueryWindow
disp = displaydatabases.Display()
disp.displayDatabases

## Talking to a Database using SQL

### Selecting records from one or more tables in a database

>**Select statement** is used to **retrieve data from a Database** 

The general **syntax*** for **select statement** is 

```mysql
select column1,column2..columnN 
from table_name
```

Where column1, column2 are the columns that you want to select from the table (with name table_name).

***Syntax** of a programming language is a set of punctuation rules that defines what the combination of symbols and characters means to the computer. 

#### Select All Columns from a Table

The syntax for selecting all columns from a table is

```mysql
select * 
from table_name
```
The <b>*</b> symbol indicates that we would want all the columns from the table. 

Let's make an example using our `film` table. 



**Query: Select all columns from `film` table**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [88]:
button1001 = Button(description="Reveal the SQL code!")
Box1001 = HBox([button1001])
que1001 = """
select *
from film;
"""
def on_click1001(b):
    clear_output()
    display(Box1001)
    return print(que1001)

button1001.on_click(on_click1001)
display(Box1001)


In [89]:
from IPython.display import display
import displaydatabases
from questiondisplay import QueryWindow

display(QueryWindow(1001).display())


#### Now let's see how to select a *subset* of columns from a table

The syntax for selecting subset of columns from a table is

```mysql
select column1,column2...columnN 
from table_name
```
Let's look at a concrete example.


<img src = "supplementary/images/SELECT_COLUMN.png" width = "500px">

The query
```mysql
select name 
from BigCats
```
will retrieve the single column 'name' and the 5 records associated with it.

Try out yourself in the next slides!

**Query: Select film title, release year, and duration from `film` table.**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [90]:
button1002 = Button(description="Reveal the SQL code!")
Box1002 = HBox([button1002])
que1002 = """
select title, release_year, length 
from film;
"""
def on_click1002(b):
    clear_output()
    display(Box1002)
    return print(que1002)

button1002.on_click(on_click1002)
display(Box1002)


In [91]:
display(QueryWindow(1002).display())

#### What if we want to select distinct values from a column?

The **syntax** for **select distinct statement** is 

```mysql
select distinct column1,column2..columnN 
from table_name
```

Where column1, column2, columnN are the columns that you want to select from the table (with name table_name) and only distinct values for column1 will be selected

Let's see a concrete example.


<img src = "supplementary/images/distinct.png" width = "500px">

The query
```mysql
select distinct state 
from Salary
```
will retrieve the unique states from the state column

Use `distinct` keyword to make a query yourself in the next slide!

**Query: Select unique ratings from `film` table**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [92]:
button1003 = Button(description="Reveal the SQL code!")
Box1003 = HBox([button1003])
que1003 = """
select distinct rating 
from film;
"""
def on_click1003(b):
    clear_output()
    display(Box1003)
    return print(que1003)

button1003.on_click(on_click1003)
display(Box1003)


In [93]:
display(QueryWindow(1003).display())

## Now Let's Learn to Filter Records by Condition! BAAM!!!

The **where clause** is used to extract records that meets a specified condition. It is one of the most powerful feature of an SQL query.

We usually use comparison operators such as `=`, `>`, `<`, `>=`, `<=` , and `!=` (i.e., not equal to) to make where-clauses. 

The syntax for where clause is 

```mysql
SELECT column1, column2, ..columnN
FROM table_name
WHERE condition;
```

Let's look at some concrete examples

The following query will fetch all rows and columns (since we are using \*) that match the condition of salary being greater than 200000.

```mysql
select * 
from Salary 
where salary>200000
```
<img src = "supplementary/images/whereI.png" width = "500px">

Let's query from our film table in the next slide. 

**Query: From film table select films having length more than 100 minutes**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [94]:
button6 = Button(description="Reveal the SQL code!")
Box6 = HBox([button6])
que6 = """
select * 
from film 
where length>100;
"""
def on_click6(b):
    clear_output()
    display(Box6)
    return print(que6)

button6.on_click(on_click6)
display(Box6)


In [95]:
display(QueryWindow(6).display())

## Logical operators `AND`, `OR`, and `NOT` for filtering based on multiple conditions 

The AND and OR operators are used for filtering records based on more than one condition.  <br/>
The AND operator displays a record if **all the conditions separated by AND are True**

The **syntax for AND operator** is 

```mysql
SELECT column1, column2, ..columnN
FROM table_name
WHERE condition1 AND condition2 AND condition3...
```

The OR operator displays a record if **any of the conditions separated by OR is True**.

The **syntax for OR operator** is 

```mysql
SELECT column1, column2, ..columnN
FROM table_name
WHERE condition1 OR condition2 OR condition3...
```

The NOT operator displays a record if the **condition(s) is NOT TRUE**.

The **syntax for NOT operator** is 

```mysql
SELECT column1, column2, ..columnN
FROM table_name
WHERE NOT condition;
```
Lets look at some examples

Using the following query we will retrieve all rows and columns that match the criteria where state is 'New York' and has a salary greater than 200000

```mysql
select * 
from Salary 
where state = 'New York' and salary>200000
```

<img src = "supplementary/images/AND.png" width = "700px">

Make a few queries from our `film` table in the next slides!

**Query: Retrieve all films from film table with length greater than 100 minutes and rating equals to PG**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [96]:
button10 = Button(description="Reveal the SQL code!")
Box10 = HBox([button10])
que10 = """
select * 
from film 
where length>100 and rating='PG';
"""
def on_click10(b):
    clear_output()
    display(Box10)
    return print(que10)

button10.on_click(on_click10)
display(Box10)




In [97]:
display(QueryWindow(10).display())

**Query: Select all films with duration greater than 80 and less than 90 from `film` table.**
* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [98]:
button1004 = Button(description="Reveal the SQL code!")
Box1004 = HBox([button1004])
que1004 = '''
select * 
from film 
where length>80 and length<90;
'''
def on_click1004(b):
    clear_output()
    display(Box1004)
    return print(que1004)

button1004.on_click(on_click1004)
display(Box1004)




In [99]:
display(QueryWindow(1004).display())

**Query: Select all films that are rated other than G.**

In [100]:
button1005 = Button(description="Reveal the SQL code!")
Box1005 = HBox([button1005])
que1005 = '''
select * 
from film 
where rating != 'G'
'''
def on_click1005(b):
    clear_output()
    display(Box1005)
    return print(que1005)

button1005.on_click(on_click1005)
display(Box1005)




In [101]:
display(QueryWindow(1005).display())

## Order records using `Order By`

Order by keyword is used to **sort query results by ascending or descending**

The syntax for Order by is as follows.
For ascending ordering:

```mysql
SELECT column1, column2, ..columnN
FROM table_name
ORDER BY column1, column2, ... ASC;
```
For descending ordering:
```mysql
SELECT column1, column2, ..columnN
FROM table_name
ORDER BY column1, column2, ... DESC;
```
Lets look at an example


<img src = "supplementary/images/desc.png" width = "700px">

The query
```mysql
select * from Salary order by salary desc
```
will retrieve all rows and columns and order them by salary in decreasing order (highest first).

If you want to order the table by salary in ascending order (lowest first), the query will be

```mysql
select * from Salary order by salary asc
```

or you can even write

```mysql
select * from Salary order by salary
```

As by default the ordering will be in ascending.

Try out the example given in the next slide.

**Query: Select film titles along with their lengths from the `film` table sorted by length descending**

In [102]:
button1006 = Button(description="Reveal the SQL code!")
Box1006 = HBox([button1006])
que1006 = '''
select title, length 
from film 
order by length desc;
'''
def on_click1006(b):
    clear_output()
    display(Box1006)
    return print(que1006)

button1006.on_click(on_click1006)
display(Box1006)



In [103]:
display(QueryWindow(1006).display())

## Using Aggregate Functions: Finding Minimum, Maximum, Average, Sum and Count for Columns

Aggregate functions **min(), max(), avg(), sum(), and count()** can be used to find the Minimum, Maximum, Average, Sum and Count for a selected column

Syntax for **min()**

```mysql
SELECT MIN(column_name) FROM table_name
```

Syntax for **max()**

```mysql
SELECT MAX(column_name) FROM table_name
```

Syntax for **avg()**

```mysql
SELECT AVG(column_name) FROM table_name
```

Syntax for **sum()**

```mysql
SELECT sum(column_name) FROM table_name
```

Syntax for **count()**

```mysql
SELECT count(column_name) FROM table_name
```

Next slide illustrates the use of `count()` function.


`count(*)` returns the number of rows returned by the select statement. In this example the query

```mysql
SELECT count(*) 
FROM Salary 
where salary>200000
```

will return a column with name count(\*) with a single row having value of 2 (because there are only 2 records with salary greater than 200000). 
<img src = "supplementary/images/count.png" width = "700px">

If the query was

```mysql
SELECT count(*) 
FROM Salary 
where state = 'Ohio'
```

the value will be 1

Now let's look at an example for min() function

The `min()` function can be used like below:

```mysql
SELECT min(salary) 
FROM Salary
```

Returns the minimum value from the salary column in the Salary table (which is 110000).
<img src = "supplementary/images/min.png" width = "700px">
Try out yourself in the next slide. 

**Query: Select the minimum movie length from the `film` table.**

In [104]:
button1007 = Button(description="Reveal the SQL code!")
Box1007 = HBox([button1007])
que1007 = '''
select min(length)
from film;
'''
def on_click1007(b):
    clear_output()
    display(Box1007)
    return print(que1007)

button1007.on_click(on_click1007)
display(Box1007)



In [105]:
display(QueryWindow(1007).display())

## Grouping records together using `Group By` statement

The Group By statement is used to **arrange identical data into groups**. The group by clause follows the where clause (if it's present) and precedes the order by clause (if it's present). For example you want to aggregate the total number of invoices by Country, or you want to get the count of employees with different Title (how many General Manager, Sales Manager etc)

The Group By statement is often used with aggregate functions (COUNT(), MAX(), MIN(), SUM(), AVG()) to group the result-set by one or more columns.

The syntax for Group By statement

```mysql
SELECT column_name(s)
FROM table_name
WHERE condition
GROUP BY column_name(s)
ORDER BY column_name(s)
```

The where condition and order by are optional and depends up on the use-case.

Let's look at some examples


The following query groups by the column State and then applys the aggregate function `count()`
 

```mysql
SELECT State,count(*) 
from Salary 
group by State
```

You can think of `count(*)` as being applied to each set of states grouped together. We have four different states with two states having count 2 and 2 states having count of 1. 

<img src = "supplementary/images/groupbycount.png" width = "700px">

**Point to Ponder**

This will give just the state name and the count. What if we were able to Map the counts. Then we could see whether there are any spatial patterns. This is the power of spatial data. The ability to map gives you the flexibility to look at the data from a completely new perspective. We will look at how this is possible in the next segment.

And by this query

```mysql
SELECT Job_Title,min(salary) 
from Salary 
group by Job_Title
```
we select the minimum salary for each Job_Title.
<img src = "supplementary/images/groupbymin.png" width = "700px">
Now make a query using our `film` table. 

**Query: Count the number of movies for each rating type from the `film` table.**

In [106]:
button1008 = Button(description="Reveal the SQL code!")
Box1008 = HBox([button1008])
que1008 = '''
select rating, count(title) 
from film 
group by rating;
'''
def on_click1008(b):
    clear_output()
    display(Box1008)
    return print(que1008)

button1008.on_click(on_click1008)
display(Box1008)



In [107]:
display(QueryWindow(1008).display())

## Aliases for providing Temporary Name

Aliases are used to give a **table or column in a table a temporary name**. Most of the time aliases are used to make the **query more readable**. Aliases **only exists until the query is running**.  

The syntax for column alias is 

```mysql
SELECT column_name AS alias_name
FROM table_name;
```

And syntax for table alias is 

```mysql
SELECT column_name
FROM table_name as alias_name;
```

Let's look at some examples

### Column Aliasing
In this query

```mysql
SELECT Job_Title,min(salary) as MIN_SALARY 
from Salary 
group by Job_Title
```
we are using an alias name for the column min(salary) (which is part of the result) as MIN_SALARY. Eventhough it doesn't change anything to the result, it makes the result more readable. 
<img src = "supplementary/images/columnalias.png" width = "700px">
Next we look at how we can use table alias in an effective way. 

### Table Aliasing
In this query 

```mysql
SELECT s.Job_Title,s.salary from Salary as s where s.state
```
we are setting an alias name 's' for the table Salary. Notice how we use the '.' operator to access the columns. This is particularly useful when we are joining multiple tables (about which we will learn in the last section). 
<img src = "supplementary/images/groupbyavg.png" width = "700px">
Try out aliasing in the next slide. 

**Query: Select the average rental_rate for each rating for each release year from the `film` table. Name the average rental_rate column as `Average_Rental`.**

In [108]:
button1009 = Button(description="Reveal the SQL code!")
Box1009 = HBox([button1009])
que1009 = '''
select rating, avg(rental_rate) as Average_Rental 
from film 
group by rating;
'''
def on_click1009(b):
    clear_output()
    display(Box1009)
    return print(que1009)

button1009.on_click(on_click1009)
display(Box1009)



In [109]:
display(QueryWindow(1009).display())

## Join for Combining Multiple Tables

A join clause is used to **combine records from multiple tables using related columns between them.** It is one of the most powerful concepts in a relational database (joining based on relations). There are four ways of combining tables

1. Inner Join
2. Left Join
3. Right Join
4. Outer Join

We will be covering only Inner Join here

## Inner Join
For inner join **records that have matching values in both tables will only be retrieved**.  

![innerjoin](supplementary/images/innerjoin.png)

The syntax for inner join is 

```mysql
SELECT column_name(s)
FROM table1
INNER JOIN table2
ON table1.column_name = table2.column_name
```

Inner join can also be written without an explicit inner join clause

```mysql
SELECT column_name(s)
FROM table1,table2
where table1.column_name = table2.column_name
```

We ill be mostly using the second form of inner join for this section.

Let's look at an example based on the customer and orders tables in the next slide.

Look at the tables below and select customers name along with the number of orders they made.
<img src = "supplementary/images/join.png" width = "500px">


```mysql
select c.name,count(*) as total_orders 
from customer as c,orders as o 
where c.CustomerId = o.CustId 
group by c.name
```

This should return


<img src = "supplementary/images/join_res1.png" width = "300px">


To get a better understanding let's see a pictorial representation of how this all works.

When you are joining the two tables using a **where clause (c.CustomerId = o.CustId)** we are getting a result with the matching records from both the tables

<img src = "supplementary/images/join_partials.png" width = "1000px">
Now when you apply the group by name and count(\*) you get the total number of records for each unique name. 

If you simply write the query

```mysql
select * 
from customer as c, orders as o 
where c.CustomerId = o.CustId
```

you will get the table just shown above.

Look at the syntax for the query. We are using two tables customer and orders. We seperate them by ','. We also know that CustomerId is the primary key for Customer table and its a foreign key in orders table with the name CustId. We use this relationship in our **where** clause to join the tow tables. Notice that we have used alias name 'c' and 'o' for customer and orders table respectively. 

Let's get back to our film and actor tables in the next slide! 


Use the textbox below to take a look at tables `film`, `actor`, and `film_actor`and try to identify the common columns that we can use as primary and foreign keys to join them.

* hint 1: `film_id` from `film` table, `actor_id` from `actor` table are the keys that are related in the `film_actor` table. 

* hint 2: Please note that the `LIMIT` function (displayed in the textbox below) truncates the table for view to the number of rows you specify. 

In [110]:

connection = sqlite3.connect(f'databases/sqlite-sakila.db')
cursor = connection.cursor()
inp1010 = Textarea(description='<b>Query:</b>', value='SELECT * \nFROM film_actor \nLIMIT 4', layout=Layout(width='40%', height='120px'))
button1010 = Button(description="Execute!")
Box1010 = HBox([inp1010, button1010])

def execute_query1010(b):
    q = cursor.execute(inp1010.value)
    f = q.fetchall()
    names = list(map(lambda x: x[0], cursor.description))
    out = pd.DataFrame(f, columns = names)
    clear_output()
    button1010.on_click(execute_query1010)
    display(Box1010)
    return display(out)

button1010.on_click(execute_query1010)
display(Box1010)

### Now that you identified the key columns let's query from the combination of the three tables.

**Query: Select title and actor first_name and actor last_name for all the films from film, actor and film_actor tables**

* hint: You can see the correct SQL code by clicking on the `Reveal the SQL code!` button. 

In [111]:
button36 = Button(description="Reveal the SQL code!")
Box36 = HBox([button36])
que36 = '''
SELECT title, first_name, last_name
FROM film_actor fa, film f, actor a
WHERE f.film_id = fa.film_id and fa.actor_id=a.actor_id
'''
def on_click36(b):
    clear_output()
    display(Box36)
    return print(que36)

button36.on_click(on_click36)
display(Box36)


In [112]:
display(QueryWindow(36).display())

## What's missing?...
Now that you have a good idea about writing SQL queries, what do you think is missing!!!

Let's again look at the Kindergarten and Shootings tables

<img src = "supplementary/images/kinder_shooting.png" width = "600px">
What if we want to ask questions such as

1. Which are the schools that have atleast a single shooting event **with in 100 meters** of its location

2. How many shooting events **with in 1000 meters** of each schools. 

3. How many shooting events **with in each census tracts**.

4. How many shooting events **with in each zip codes**.

5. The **distance** to the closest shooting event for each schools.

Can we answer such questions based on our tables. We seems to have a location field, but currently its just text and we can't ask any spatial questions using Text data type. So how will we solve this.

The answer to these questions will be clear when we learn about spatial databases and spatial queries in the next section. So stay tuned!!!!! 

Click the link below to move on


<br>
<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="gd-5.ipynb">Click here to go to the next notebook.</a></font>